In [4]:
from ortools.sat.python import cp_model

var_range = 500

# Create the model
model = cp_model.CpModel()

# Define integer variables for edge lengths a, b, c, d, e, f
a = model.NewIntVar(1, var_range, "a")
b = model.NewIntVar(1, var_range, "b")
c = model.NewIntVar(1, var_range, "c")
d = model.NewIntVar(1, var_range, "d")
e = model.NewIntVar(1, var_range, "e")
f = model.NewIntVar(1, var_range, "f")

# Create variables to represent the squares of the edge lengths
a_squared = model.NewIntVar(1, var_range * var_range, "a_squared")
b_squared = model.NewIntVar(1, var_range * var_range, "b_squared")
c_squared = model.NewIntVar(1, var_range * var_range, "c_squared")
d_squared = model.NewIntVar(1, var_range * var_range, "d_squared")
e_squared = model.NewIntVar(1, var_range * var_range, "e_squared")
f_squared = model.NewIntVar(1, var_range * var_range, "f_squared")

# Add constraints to define the squares of the lengths
model.AddMultiplicationEquality(a_squared, [a, a])
model.AddMultiplicationEquality(b_squared, [b, b])
model.AddMultiplicationEquality(c_squared, [c, c])
model.AddMultiplicationEquality(d_squared, [d, d])
model.AddMultiplicationEquality(e_squared, [e, e])
model.AddMultiplicationEquality(f_squared, [f, f])

# Define intermediate variables X, Y, Z
X = model.NewIntVar(-var_range * var_range * 2, var_range * var_range * 2, "X")
Y = model.NewIntVar(-var_range * var_range * 2, var_range * var_range * 2, "Y")
Z = model.NewIntVar(-var_range * var_range * 2, var_range * var_range * 2, "Z")

# Add equality constraints for X, Y, Z
model.Add(X == b_squared + c_squared - d_squared)
model.Add(Y == a_squared + c_squared - e_squared)
model.Add(Z == a_squared + b_squared - f_squared)

# Define intermediate variables for products
a2b2 = model.NewIntVar(1, var_range**4, "a2b2")
a2c2 = model.NewIntVar(1, var_range**4, "a2c2")
b2c2 = model.NewIntVar(1, var_range**4, "b2c2")
a2X = model.NewIntVar(-(var_range**4), var_range**4, "a2X")
a2X2 = model.NewIntVar(1, 2 * var_range**4, "a2X2")
b2Y = model.NewIntVar(-(var_range**4), var_range**4, "b2Y")
b2Y2 = model.NewIntVar(1, 2 * var_range**4, "b2Y2")
c2Z = model.NewIntVar(-(var_range**4), var_range**4, "c2Z")
c2Z2 = model.NewIntVar(1, 2 * var_range**4, "c2Z2")
XY = model.NewIntVar(-(var_range**4), var_range**4, "XY")
XYZ = model.NewIntVar(-(var_range**6), var_range**6, "XYZ")
a2b2c2 = model.NewIntVar(1, var_range**6, "a2b2c2")
four_a2b2c2 = model.NewIntVar(1, var_range**6, "four_a2b2c2")
# Define intermediate variables for the Heron's formula constraint
s = model.NewIntVar(1, 3 * var_range, "s")
s_a = model.NewIntVar(1, 3 * var_range, "s_a")
s_b = model.NewIntVar(1, 3 * var_range, "s_b")
s_c = model.NewIntVar(1, 3 * var_range, "s_c")
heron_product = model.NewIntVar(1, var_range**3, "heron_product")

# Add constraints for Heron's formula
model.Add(s == a + b + c)
model.Add(s_a == s - a)
model.Add(s_b == s - b)
model.Add(s_c == s - c)
model.AddMultiplicationEquality(heron_product, [s, s_a, s_b, s_c])

# Add the Heron's formula constraint
model.Add(heron_product == 32).OnlyEnforceIf(model.NewBoolVar("heron_32"))
model.Add(heron_product == 128).OnlyEnforceIf(model.NewBoolVar("heron_128"))
model.AddMultiplicationEquality(a2b2, [a_squared, b_squared])
model.AddMultiplicationEquality(a2c2, [a_squared, c_squared])
model.AddMultiplicationEquality(b2c2, [b_squared, c_squared])
model.AddMultiplicationEquality(a2X, [a_squared, X])
model.AddMultiplicationEquality(a2X2, [a2X, X])
model.AddMultiplicationEquality(b2Y, [b_squared, Y])
model.AddMultiplicationEquality(b2Y2, [b2Y, Y])
model.AddMultiplicationEquality(c2Z, [c_squared, Z])
model.AddMultiplicationEquality(c2Z2, [c2Z, Z])
model.AddMultiplicationEquality(XY, [X, Y])
model.AddMultiplicationEquality(XYZ, [XY, Z])
model.AddMultiplicationEquality(a2b2c2, [a2b2, c_squared])
model.AddMultiplicationEquality(four_a2b2c2, [a2b2c2, 4])

# Define the volume squared constraint using the new formula
volume_squared = model.NewIntVar(0, 1000000, "volume_squared")
model.Add(volume_squared == (four_a2b2c2 - a2X2 - b2Y2 - c2Z2 + XYZ))

# Set the target volume squared constraint to match the requirement
model.Add(volume_squared == 128)  # Adjust the target value as needed

# Add triangle inequalities to ensure valid tetrahedron sides
model.Add(a + b > c)
model.Add(a + c > b)
model.Add(b + c > a)
model.Add(a + d > e)
model.Add(a + e > d)
model.Add(d + e > a)
model.Add(b + d > f)
model.Add(b + f > d)
model.Add(d + f > b)
model.Add(c + e > f)
model.Add(c + f > e)
model.Add(e + f > c)

model.Add(a <= b)
model.Add(b <= c)
model.Add(c <= d)
model.Add(d <= e)
model.Add(e <= f)

# Exclude the trivial solution [2, 2, 2, 2, 2, 2]
model.Add(a != 2)
# model.Add(b != 2)
# model.Add(c != 2)
# model.Add(d != 2)
# model.Add(e != 2)
# model.Add(f != 2)

# Solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Output solution if found
if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
    a_val = solver.Value(a)
    b_val = solver.Value(b)
    c_val = solver.Value(c)
    d_val = solver.Value(d)
    e_val = solver.Value(e)
    f_val = solver.Value(f)

    print("Solution:")
    print(f"a = {a_val}, b = {b_val}, c = {c_val},")
    print(f"d = {d_val}, e = {e_val}, f = {f_val}")

    # Verify the volume constraint
    a2 = a_val**2
    b2 = b_val**2
    c2 = c_val**2
    d2 = d_val**2
    e2 = e_val**2
    f2 = f_val**2

    X_val = b2 + c2 - d2
    Y_val = a2 + c2 - e2
    Z_val = a2 + b2 - f2

    volume_squared_val = (
        4 * a2 * b2 * c2
        - a2 * X_val * X_val
        - b2 * Y_val * Y_val
        - c2 * Z_val * Z_val
        + X_val * Y_val * Z_val
    )

    volume_val = volume_squared_val / 288

    print(f"Computed volume squared constraint: {volume_squared_val}")
    print(f"Computed volume constraint: {volume_val}")
    if volume_squared_val == 128:
        print("The solution satisfies the volume constraint.")
    else:
        print("The solution does not satisfy the volume constraint.")
else:
    print("No solution found.")

No solution found.


In [ ]:
a, b, c, x, y, z = 1, 2, 3, 5, 6, 4

a2 = a * a
b2 = b * b
c2 = c * c
x2 = x * x
y2 = y * y
z2 = z * z

X_val = b2 + c2 - x2
Y_val = a2 + c2 - y2
Z_val = a2 + b2 - z2

V_squared = (
    4 * a2 * b2 * c2
    - a2 * X_val * X_val
    - b2 * Y_val * Y_val
    - c2 * Z_val * Z_val
    + X_val * Y_val * Z_val
)
V_squared

-7225

In [12]:
from sympy import Symbol, expand, simplify, factor

In [3]:
a, b, c, x, y, z = Symbol("a"), Symbol("b"), Symbol("c"), Symbol("x"), Symbol("y"), Symbol("z")

In [16]:
expression_1 = 4*a**2*b**2*c**2 - a**2*(b**2 + c**2 - x**2)**2 - b**2*(a**2 + c**2 - y**2)**2 - c**2*(a**2 + b**2 - z**2)**2 + (b**2 + c**2 - x**2)*(a**2 + c**2 - y**2)*(a**2 + b**2 - z**2)
expression_2 = ((x+y)**2 - z**2)*(x-y+z)*(-x+y+z)

In [17]:
simplify(expression_1 - expression_2)

4*a**2*b**2*c**2 - a**2*(b**2 + c**2 - x**2)**2 - b**2*(a**2 + c**2 - y**2)**2 - c**2*(a**2 + b**2 - z**2)**2 + (z**2 - (x + y)**2)*(-x + y + z)*(x - y + z) + (a**2 + b**2 - z**2)*(a**2 + c**2 - y**2)*(b**2 + c**2 - x**2)

In [18]:
factor(expand(expression_2))

-(x - y - z)*(x - y + z)*(x + y - z)*(x + y + z)

In [37]:
a, b, c, x, y, z = 2, 2, 2, 2, 2, 2

In [38]:
4*a**2*b**2*c**2 - a**2*(b**2 + c**2 - x**2)**2 - b**2*(a**2 + c**2 - y**2)**2 - c**2*(a**2 + b**2 - z**2)**2 + (b**2 + c**2 - x**2)*(a**2 + c**2 - y**2)*(a**2 + b**2 - z**2)

128

In [19]:
factor(expression_1-expression_2)

-a**4*x**2 + a**2*b**2*x**2 + a**2*b**2*y**2 - a**2*b**2*z**2 + a**2*c**2*x**2 - a**2*c**2*y**2 + a**2*c**2*z**2 - a**2*x**4 + a**2*x**2*y**2 + a**2*x**2*z**2 - b**4*y**2 - b**2*c**2*x**2 + b**2*c**2*y**2 + b**2*c**2*z**2 + b**2*x**2*y**2 - b**2*y**4 + b**2*y**2*z**2 - c**4*z**2 + c**2*x**2*z**2 + c**2*y**2*z**2 - c**2*z**4 + x**4 - x**2*y**2*z**2 - 2*x**2*y**2 - 2*x**2*z**2 + y**4 - 2*y**2*z**2 + z**4